In [2]:
# ONLY HAS TO BE RUN ONCE TO EXPORT DATASET FROM ZIP TO FOLDER
import zipfile

zip_path = "Resources.zip" 
extract_to = "Dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [1]:
# ONLY HAS TO BE RUN ONCE TO ENABLE 5-FOLD CROSS-VALIDATION
import os
import shutil

# Making a dictionary to store the disease class of each patient
dataset_path = "Dataset/training"  # Path to train + validation folder
patient_class_dict = {}

for patient_folder in os.listdir(dataset_path): # For file in the training folder
    if patient_folder.startswith("."):  # Skip hidden folders (.ipynb_checkpoints)
        continue

    patient_path = os.path.join(dataset_path, patient_folder) # Patient ID, such as patient001

    if os.path.isdir(patient_path):  # Process only valid patient folders (skip MANDATATORY_CITATION)
        info_file = os.path.join(patient_path, "Info.cfg")
        with open(info_file, "r") as patient_file: # Open file
                lines = patient_file.readlines()
                patient_class = lines[2].strip()
                patient_class_dict[patient_folder] = patient_class # Add to dictionary
        
# Splitting dataset based on dictionary values (20 patients in each dataset)
group_DCM = "Dataset/group_DCM"
group_HCM = "Dataset/group_HCM"
group_MINF = "Dataset/group_MINF"
group_NOR = "Dataset/group_NOR"
group_RV = "Dataset/group_RV"

# Create the directories for each class
for group in [group_DCM, group_HCM, group_MINF, group_NOR, group_RV]:
    if not os.path.exists(group):
        os.makedirs(group)

# Loop through all patients
for patient_folder, disease in patient_class_dict.items():
    patient_path = os.path.join(dataset_path, patient_folder)

    # Check if the folder exists and it's a directory
    if os.path.isdir(patient_path):
        # Determine the target group based on disease
        if "DCM" in disease:
            target_group = group_DCM
        elif "HCM" in disease:
            target_group = group_HCM
        elif "MINF" in disease:
            target_group = group_MINF
        elif "NOR" in disease:
            target_group = group_NOR
        elif "RV" in disease:
            target_group = group_RV
        else:
            print('unknown class error')
            continue

        # Create the patient's folder inside the target group directory
        target_patient_folder = os.path.join(target_group, patient_folder)
        if not os.path.exists(target_patient_folder):
            os.makedirs(target_patient_folder)

        # Copy respective files to new folder
        for file_name in os.listdir(patient_path):
            file_path = os.path.join(patient_path, file_name)
            if os.path.isfile(file_path):  # Check if it's a file
                # Move the file to the respective patient folder in the group folder
                shutil.copy(file_path, os.path.join(target_patient_folder, file_name)) # YOU CAN ONLY RUN THIS ONCE, AFTER THAT THE TRAINING SET IS EMPTY


In [2]:
## ALL FUNCTIONs NEEDED TO TRAIN AND VALIDATE MODEL
import os
import shutil
data_path_train = "Dataset/new_training"
data_path_valid = "Dataset/new_validation"
data_path_test = "Dataset/testing"

import glob
import nibabel as nib
import numpy as np
import monai
import torch.nn.functional as F
from medpy.metric.binary import hd, dc
import time
import torch
from monai.data import CacheDataset, DataLoader
from monai.networks.nets import UNet
from monai.losses import DiceLoss
from monai.metrics import DiceMetric

# Start up wandb and start logging
import wandb
print("Done Importing!")

def get_ed_es_frames(config_path):
    """Extract ED and ES frame numbers from the info.cfg file."""

    ed_frame, es_frame = None, None
    with open(config_path, 'r') as f:
        for line in f:
            if line.startswith('ED:'):
                ed_frame = int(line.split(':')[1].strip())
            elif line.startswith('ES:'):
                es_frame = int(line.split(':')[1].strip())
    return ed_frame, es_frame


def build_dict_acdc(data_path, mode='train'):
    """
    This function returns a list of dictionaries, each containing the paths to the 2D slices 
    of the 3D MRI images and their corresponding masks.
    """
    if mode not in ["train", "val", "test"]:
        raise ValueError(f"Please choose a mode in ['train', 'val', 'test']. Current mode is {mode}.")
    
    dicts = []
    
    # Loop over all patient directories
    patient_dirs = [d for d in glob.glob(os.path.join(data_path, '*')) if os.path.isdir(d)]
    
    for patient_dir in patient_dirs:
        patient_id = os.path.basename(patient_dir)
        config_path = os.path.join(patient_dir, "Info.cfg")
        
        if not os.path.exists(config_path):
            continue
        
        ed_frame, es_frame = get_ed_es_frames(config_path)
        
        # Identify the ED and ES image and mask paths
        ed_img_path = os.path.join(patient_dir, f"{patient_id}_frame{ed_frame:02d}.nii.gz")
        ed_mask_path = os.path.join(patient_dir, f"{patient_id}_frame{ed_frame:02d}_gt.nii.gz")
        es_img_path = os.path.join(patient_dir, f"{patient_id}_frame{es_frame:02d}.nii.gz")
        es_mask_path = os.path.join(patient_dir, f"{patient_id}_frame{es_frame:02d}_gt.nii.gz")
        
        for img_path, mask_path in [(ed_img_path, ed_mask_path), (es_img_path, es_mask_path)]:
            if not os.path.exists(img_path) or not os.path.exists(mask_path):
                continue
            
            # Load the 3D image and mask using nibabel
            img_volume = nib.load(img_path).get_fdata()
            mask_volume = nib.load(mask_path).get_fdata()
            #print("Unique values in loaded ground truth mask:", np.unique(mask_volume))
            
            # Ensure we have the same number of slices for image and mask
            num_slices = img_volume.shape[2]
            
            # Extract 2D slices
            for slice_idx in range(num_slices):
                img_slice = img_volume[:, :, slice_idx]
                mask_slice = mask_volume[:, :, slice_idx]
                
                dicts.append({'img': img_slice, 'mask': mask_slice})
    
    return dicts

class LoadHeartData(monai.transforms.Transform):
    """
    This custom Monai transform loads 2D slices of MRI data and their corresponding mask for heart segmentation.
    """
    def __init__(self, keys=None):
        pass
    
    def __call__(self, sample):
        img_slice = sample['img']
        mask_slice = sample['mask'] 
        
        # Ensure the image and mask are in compatible formats
        img_slice = np.array(img_slice, dtype=np.float32)
        mask_slice = np.array(mask_slice, dtype=np.uint8) 
        
        # Return the slice and mask with metadata. NOT SURE ABOUT THE METATDATA
        return {'img': img_slice, 'mask': mask_slice, 'img_meta_dict': {'affine': np.eye(2)}, 
                'mask_meta_dict': {'affine': np.eye(2)}}

HEADER = ["Name", "Dice LV", "Volume LV", "Err LV(ml)",
          "Dice RV", "Volume RV", "Err RV(ml)",
          "Dice MYO", "Volume MYO", "Err MYO(ml)"]


# Functions to process files, directories and metrics aka loss function

def metrics(img_gt, img_pred, voxel_size):
    """
    Function to compute the metrics between two segmentation maps given as input.

    Return
    ------
    A list of metrics in this order, [Dice LV, Volume LV, Err LV(ml),
    Dice RV, Volume RV, Err RV(ml), Dice MYO, Volume MYO, Err MYO(ml)]
    """

    if img_gt.ndim != img_pred.ndim:
        raise ValueError("The arrays 'img_gt' and 'img_pred' should have the "
                         "same dimension, {} against {}".format(img_gt.ndim,
                                                                img_pred.ndim))
    
    res = []
    
    # Loop on each classes of the input images
    for c in [3, 1, 2]:
        # Copy the gt image to not alterate the input
        gt_c_i = np.copy(img_gt)
        gt_c_i[gt_c_i != c] = 0

        # Copy the pred image to not alterate the input
        pred_c_i = np.copy(img_pred)
        pred_c_i[pred_c_i != c] = 0

        # Clip the value to compute the volumes
        gt_c_i = np.clip(gt_c_i, 0, 1)
        pred_c_i = np.clip(pred_c_i, 0, 1)

        # Compute the Dice
        dice = dc(gt_c_i, pred_c_i)

        # Compute volume
        volpred = pred_c_i.sum() * np.prod(voxel_size) / 1000.
        volgt = gt_c_i.sum() * np.prod(voxel_size) / 1000.

        res += [dice, volpred, volpred-volgt]

    return res

def compute_metrics_on_files(path_gt, path_pred):
    """
    Function to give the metrics for two files

    """
    gt, _, header = load_nii(path_gt)
    pred, _, _ = load_nii(path_pred)
    zooms = header.get_zooms()

    name = os.path.basename(path_gt)
    name = name.split('.')[0]
    res = metrics(gt, pred, zooms)
    res = ["{:.3f}".format(r) for r in res]

    formatting = "{:>14}, {:>7}, {:>9}, {:>10}, {:>7}, {:>9}, {:>10}, {:>8}, {:>10}, {:>11}"
    print(formatting.format(*HEADER))
    print(formatting.format(name, *res))
      
# Recombine into a training and a validation set (set 1 to validation and 4 to training)
def recombining_data(recombine_index):
    
    new_train_path = os.path.join("Dataset", 'new_training')
    new_val_path = os.path.join("Dataset", 'new_validation')
    
    # If folder does not exist yet
    if not os.path.exists(new_train_path):
        os.makedirs(new_train_path)  # Creates the new training folder
    if not os.path.exists(new_val_path):
        os.makedirs(new_val_path)  # Creates the new validation folder
    
    # Empty the new_validation folder
    if os.path.exists(new_val_path):
        shutil.rmtree(new_val_path)
        os.makedirs(new_val_path)

    # Empty the new_training folder
    if os.path.exists(new_train_path):
        shutil.rmtree(new_train_path)
        os.makedirs(new_train_path)
    
    val_id = [1,2,3,4]
    offset = (recombine_index - 1) * 4
    val_id = [element + offset for element in val_id] # Add the offset to each element of val_id
    
    train_id = list(range(1,21))
    for element in val_id:
        train_id.remove(element) # remove the validation patients
    
    # now there is a list of numbers for who should be in val, and who should be in train
    
    # Define the classes (group folders) you want to loop through
    class_folders = ['group_DCM', 'group_HCM', 'group_MINF', 'group_NOR', 'group_RV']
    
    # Loop through each class folder
    for class_folder in class_folders:
        class_folder_path = os.path.join("Dataset", class_folder)
        
        patients_in_class = [folder for folder in os.listdir(class_folder_path)] # list of all file names in class
        
        for val_target in val_id: # copy all validation patients
            val_patient_target = patients_in_class[val_target - 1] # get name of validation patient
            
            # copy from source to destination
            source_folder = os.path.join("Dataset", class_folder,val_patient_target)
            destination_folder = os.path.join(new_val_path, val_patient_target)
            shutil.copytree(source_folder, destination_folder)
            
        for train_target in train_id: # copy all training patients
            train_patient_target = patients_in_class[train_target - 1] # get name of training patient
            
            # copy from source to destination
            source_folder = os.path.join("Dataset", class_folder,train_patient_target)
            destination_folder = os.path.join(new_train_path, train_patient_target)
            shutil.copytree(source_folder, destination_folder)


2025-04-12 17:13:57.174759: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-12 17:13:57.211732: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-12 17:13:57.211752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-12 17:13:57.211772: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-12 17:13:57.218965: I tensorflow/core/platform/cpu_feature_g

Done Importing!


In [3]:
# ENTIRE TRAINING AND MODEL SAVING LOOP, sending all training, validation and test data to wandb
from monai.transforms import (
    Compose,
    LoadImaged,
    AddChanneld,
    ScaleIntensityd,
    Spacingd,
    ResizeWithPadOrCropd,
    EnsureTyped,
    RandZoomd,
    RandFlipd,
    RandRotated,
)

experiment_name = "PreProc_2DUNet_Cross_validation" # CHANGE THIS PER RUN!

# Define folder based on the WandB run name and create the folder.
folder_save_path = experiment_name +  "_models"
os.makedirs(folder_save_path, exist_ok=True)

print("Beginning the loop")
# Recombine dataset
recombine_index = [1, 2, 3, 4, 5]

# Beginning Model train loop
for idx in recombine_index:
    print("Beginning loop for index:" + str(idx))
    # Recombine new validation and training set (in folders)
    print("Recombining Data")
    recombining_data(idx)

    # Initialize a new WandB run with configuration based on the experiment name.
    run = wandb.init(
        entity="DLMI_Project",
        project="DLMI_Project",
        config={
            "learning_rate": 1e-4,
            "architecture": experiment_name,  # Using experiment name as the architecture identifier
            "dataset": "ACDC",
            "epochs": 10, # Adjust to e.g. 80 for a proper training
        },
        name=f"{experiment_name}_run_{idx}"
    )

    # Combine the folder path with the model filename.
    model_save_path = os.path.join(folder_save_path, f"{experiment_name}_cross_variant_{idx}.pth")
    
    print("PyTorch version:", torch.__version__)
    print("CUDA version (PyTorch):", torch.version.cuda)

    # Define a common preprocessing pipeline (containing the deterministic transforms)
    common_transform = Compose([
        LoadHeartData(),  
        AddChanneld(keys=["img", "mask"]), # Add channel dimension
        ScaleIntensityd(keys=["img"], minv=0, maxv=1),  # Normalize intensity
        Spacingd(keys=["img", "mask"], pixdim=(1.25, 1.25), mode=("bilinear", "nearest")), # Resample voxel spacing in x and y
        ResizeWithPadOrCropd(keys=["img", "mask"], spatial_size=[256, 256]), # Ensures all images have the same dimensions (without getting stretched out). 
        EnsureTyped(keys=["img", "mask"])
    ])

    # Train Transform
    train_transforms = Compose([
        *common_transform.transforms,  # Apply all common steps first
        RandZoomd(keys=["img", "mask"], prob=0.1, min_zoom=0.9, max_zoom=1.1, keep_size=True), # Random zoom, not too much so that you don't remove important parts
        RandFlipd(keys=["img", "mask"], prob=0.1, spatial_axis=0),  # Random flip. Spatial axis=0 for up-down flipping. Left-right flipping is not good because the model has to distinguish the left and right ventricle
        RandRotated(keys=["img", "mask"], range_x=np.pi/12, prob=0.1, mode=("bilinear", "nearest")), # Random rotation for max 15 degrees
    ])

    # Validation and test transforms (only containing the deterministic transforms)
    test_transforms = common_transform
    valid_transforms = common_transform

    train_data = build_dict_acdc(data_path_train, mode='train')
    test_data = build_dict_acdc(data_path_test, mode='test')
    valid_data = build_dict_acdc(data_path_valid, mode='val')

    # Create CacheDatasets for training and testing
    train_dataset = CacheDataset(data=train_data, transform=train_transforms)
    test_dataset = CacheDataset(data=test_data, transform=test_transforms)
    valid_dataset = CacheDataset(data=valid_data, transform=test_transforms)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=4)

    # DEFINE THE ARCHITECTURE
    # ---------------------------------------------------------------------------------------------------------------
    
    # Define the device to use
    print("CUDA Available:", torch.cuda.is_available())
    print("CUDA Device Count:", torch.cuda.device_count())
    print("CUDA Current Device:", torch.cuda.current_device() if torch.cuda.is_available() else "No GPU")
    print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
    print("PyTorch version:", torch.__version__)
    print("CUDA version (PyTorch):", torch.version.cuda)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    torch.cuda.empty_cache()
    
    # Initialize the 2D U-Net model
    model = UNet(
        spatial_dims=2, # 2D 
        in_channels=1, # Grayscale
        out_channels=4, # Multi-label segmentation
        channels=(64, 128, 256, 512, 1024),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)

    wandb.watch(model, log="all")
    
    # Define the loss function and optimizer.
    loss_function = DiceLoss(softmax=True) # DiceLoss with softmax=True for multilabel segmentation.
    optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate) 

    # (Optional) DiceMetric for evaluation during training
    dice_metric = DiceMetric(include_background=True, reduction="mean")

    print("Model loaded")
    print(torch.__version__)

    # START THE TRAINING
    # --------------------------------------------------------------------------------------------------------------------------
    
    # Training loop
    num_epochs = wandb.config.epochs
    for epoch in range(num_epochs):
        print("-" * 10, f"Epoch {epoch + 1}/{num_epochs}", "-" * 10)
        model.train()
        epoch_loss = 0
        step = 0
        start_time = time.time()

        for batch_data in train_loader:
            step += 1
            inputs = batch_data["img"].to(device)
            # Convert labels from shape (B, 1, H, W) to (B, H, W)
            labels = batch_data["mask"].squeeze(1).to(device)
            unique_values = torch.unique(labels)

            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)  # shape: (B, 4, H, W)
            outputs = outputs.contiguous()
            # Convert labels to one-hot encoding: shape becomes (B, H, W, 4)
            one_hot_labels = F.one_hot(labels.long(), num_classes=4)
            # Permute to get shape (B, 4, H, W)
            one_hot_labels = one_hot_labels.permute(0, 3, 1, 2).float()

            # Compute the loss
            loss = loss_function(outputs, one_hot_labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            wandb.log({"train_step_loss": loss.item(), "epoch": epoch + 1})

        epoch_loss /= step
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch + 1} average loss: {epoch_loss:.4f}, time: {epoch_time:.2f} sec")

        # Validation
        voxel_size = (1.25, 1.25)  # For 2D slices; adjust as needed

        model.eval()
        all_metrics = []
        with torch.no_grad():
            for val_data in valid_loader: # switch to validation loader
                val_inputs = val_data["img"].to(device)
                val_labels = val_data["mask"].squeeze(1).to(device)  # shape: (B, H, W)
                val_outputs = model(val_inputs)  # shape: (B, 4, H, W)

                # For evaluation, use the integer label maps directly.
                pred_labels = torch.argmax(torch.softmax(val_outputs, dim=1), dim=1)  # (B, H, W)
                gt_labels = val_labels  # already in (B, H, W) after squeeze

                # Convert to numpy arrays
                pred_labels_np = pred_labels.cpu().numpy()
                gt_labels_np = gt_labels.cpu().numpy()

                for gt, pred in zip(gt_labels_np, pred_labels_np):
                    sample_metrics = metrics(gt, pred, voxel_size)
                    all_metrics.append(sample_metrics)

            avg_metrics = np.mean(all_metrics, axis=0)
            print("Validation metrics:", avg_metrics)
            wandb.log({
                "epoch": epoch + 1,
                "epoch_loss": epoch_loss,
                "Dice_LV": avg_metrics[0],
                "Volume_LV": avg_metrics[1],
                "Err_LV": avg_metrics[2],
                "Dice_RV": avg_metrics[3],
                "Volume_RV": avg_metrics[4],
                "Err_RV": avg_metrics[5],
                "Dice_MY0": avg_metrics[6],
                "Volume_MY0": avg_metrics[7],
                "Err_MY0": avg_metrics[8],
                "epoch_time_sec": epoch_time
            })
    

    # Save the trained model at the end
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at {model_save_path}")
    
    # Get test results
    model.eval()
    all_metrics = []
    with torch.no_grad():
        for test_data in test_loader:  
            test_inputs = test_data["img"].to(device)
            test_labels = test_data["mask"].squeeze(1).to(device)  # shape: (B, H, W)
            test_outputs = model(test_inputs)  # shape: (B, 4, H, W)

            # For evaluation, use the integer label maps directly.
            pred_labels = torch.argmax(torch.softmax(test_outputs, dim=1), dim=1)  # (B, H, W)
            gt_labels = test_labels  # already in (B, H, W) after squeeze

            # Convert to numpy arrays
            pred_labels_np = pred_labels.cpu().numpy()
            gt_labels_np = gt_labels.cpu().numpy()

            for gt, pred in zip(gt_labels_np, pred_labels_np):
                sample_metrics = metrics(gt, pred, voxel_size)
                all_metrics.append(sample_metrics)

        avg_metrics = np.mean(all_metrics, axis=0)
        print("Test metrics:", avg_metrics)
        wandb.log({
            "Dice_LV_test": avg_metrics[0],
            "Volume_LV_test": avg_metrics[1],
            "Err_LV_test": avg_metrics[2],
            "Dice_RV_test": avg_metrics[3],
            "Volume_RV_test": avg_metrics[4],
            "Err_RV_test": avg_metrics[5],
            "Dice_MY0_test": avg_metrics[6],
            "Volume_MY0_test": avg_metrics[7],
            "Err_MY0_test": avg_metrics[8]
        })

        print("Test metrics:", avg_metrics)

    run.finish()


Beginning the loop
Beginning loop for index:1
Recombining Data


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: k-h-leussink (k-h-leussink-university-of-twente) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


PyTorch version: 2.3.1+cu118
CUDA version (PyTorch): 11.8


/home/jovyan/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)
Loading dataset: 100%|██████████| 396/396 [00:04<00:00, 83.78it/s]


CUDA Available: True
CUDA Device Count: 2
CUDA Current Device: 0
CUDA Device Name: Tesla T4
PyTorch version: 2.3.1+cu118
CUDA version (PyTorch): 11.8
Using device: cuda
Model loaded
2.3.1+cu118
---------- Epoch 1/10 ----------


...

Beginning loop for index:5
Recombining Data


PyTorch version: 2.3.1+cu118
CUDA version (PyTorch): 11.8


/home/jovyan/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)
Loading dataset: 100%|██████████| 350/350 [00:04<00:00, 84.92it/s]


CUDA Available: True
CUDA Device Count: 2
CUDA Current Device: 0
CUDA Device Name: Tesla T4
PyTorch version: 2.3.1+cu118
CUDA version (PyTorch): 11.8
Using device: cuda
Model loaded
2.3.1+cu118
---------- Epoch 1/10 ----------
Epoch 1 average loss: 0.8212, time: 37.52 sec
Validation metrics: [1.69090066e-02 3.98451116e+01 3.90230491e+01 1.58038794e-01
 7.01166518e+00 6.19663393e+00 4.23886918e-01 2.12006696e+00
 1.22376786e+00]
---------- Epoch 2/10 ----------
Epoch 2 average loss: 0.7667, time: 37.78 sec
Validation metrics: [1.23596026e-02 4.56117054e+01 4.47896429e+01 4.85776902e-01
 1.52428125e+00 7.09250000e-01 5.82041387e-01 1.25500000e+00
 3.58700893e-01]
---------- Epoch 3/10 ----------
Epoch 3 average loss: 0.6911, time: 38.29 sec
Validation metrics: [ 1.50957928e-02  4.98692232e+01  4.90471607e+01  5.85514826e-01
  9.26741071e-01  1.11709821e-01  6.64641213e-01  8.86741071e-01
 -9.55803571e-03]
---------- Epoch 4/10 ----------
Epoch 4 average loss: 0.5802, time: 38.23 sec
Vali

Dice_LV,█▁▅▂▄▅▅▄▅▄
Dice_LV_test,▁
Dice_MY0,▁▄▆▆▇█▇███
Dice_MY0_test,▁
Dice_RV,▁▅▇▇██████
Dice_RV_test,▁
Err_LV,▁▅████████
Err_LV_test,▁
Err_MY0,█▄▂▂▁▂▁▁▁▁
Err_MY0_test,▁
Err_RV,█▂▁▁▁▁▁▁▁▁
